In [ ]:
import requests
import json
from pandas.io.json import json_normalize
import pandas as pd

import re
import time
from time import strptime
import datetime

import sys
sys.stdout = open('currents_output.log', 'w')

### Investigating currents 

In [ ]:
with open('current_station_info.json', 'r') as station_info_file:
    currents_station_info = json.load(station_info_file)
    
with open('current_station_intervals.json', 'r') as station_dates_file:
    currents_station_intervals = json.load(station_dates_file)
    
    

In [ ]:
for key, date_list in currents_station_info.items():
    if any(isinstance(el, list) for el in date_list):
        for i, dates in enumerate(date_list):
            for j, date in enumerate(dates[:2]):
                if date == '':
                    currents_station_info[key][0][1] = datetime.datetime.now()
                    continue
                split_date = re.findall(r"[\w']+", date)
                currents_station_info[key][i][j] = datetime.datetime(year = int(split_date[0]), 
                                month = int(split_date[1]),
                                day = int(split_date[2]), 
                                hour = int(split_date[3]), 
                                minute = int(split_date[4]))   
    else:
        currents_station_info[key] = []
        tmp_list = []
        for date in date_list[:2]:
            split_date = re.findall(r"[\w']+", date)
            tmp_list.append(datetime.datetime(year = int(split_date[2]), 
                            month = int(strptime(split_date[0],'%b').tm_mon),
                            day = int(split_date[1]), 
                            hour = int(split_date[3]),
                            minute = int(split_date[4])))
        tmp_list = tmp_list + date_list[2:]
        currents_station_info[key].append(tmp_list) #put the list inside of another list
    

In [ ]:
def retrieveLifetimeData(station_id, date_lists):
    lifetime_data = []
    for date_list in date_lists:
        begin_date = date_list[0]
        begin_date += datetime.timedelta(minutes=begin_date.minute % 6)
        
        end_date = date_list[1]
        end_date -= datetime.timedelta(minutes=end_date.minute % 6)
        
        date = begin_date
        month = datetime.timedelta(days=31)
        first_loop = True
        while 1:
            end_loop = False
            next_date = date + month
            
            if next_date > end_date: 
                next_date = end_date
                end_loop = True
                print('this is where it ends')
                
            url = 'https://tidesandcurrents.noaa.gov/api/datagetter?'
            params = {
                'begin_date': '{:02d}/{:02d}/{} {:02d}:{:02d}'.format(date.month, date.day, date.year, date.hour, date.minute),
                'end_date':'{:02d}/{:02d}/{} {:02d}:{:02d}'.format(next_date.month, next_date.day, next_date.year, next_date.hour, next_date.minute),
                'station':station_id,
                'product':'currents',
                'units':'metric',
                'time_zone':'gmt',
                'application':'web_services',
                'format':'json'    
            }
            
            bin_list = []
            
            i=1
            while 1:
                params['bin'] = i
                resp = requests.get(url=url, params=params)
                try:
                    bin_list.append(pd.DataFrame(resp.json()['data']))
                except:
                    break
                bin_list[i-1].drop('b', axis=1, inplace=True)
                bin_list[i-1].set_index('t', inplace=True)
                bin_list[i-1].rename(columns = lambda x : '{}.{}.'.format(station_id, i) + x, inplace = True)
                i += 1
            try:    
                monthly_data = pd.concat(bin_list, axis=1)
                lifetime_data.append(monthly_data)
            except ValueError:
                print('Lost data for {}  -  {}'.format(date, next_date))
                pass
            
            date = next_date
            if end_loop:
                break
    print('Done with {}'.format(station_id))            
    return pd.concat(lifetime_data)

In [ ]:
all_of_the_data = []
total = len(currents_station_info.keys())
counter = 0
for station_id, available_dates in currents_station_info.items():
    counter += 1
    print('{} of {}'.format(counter, total))
    print('{}:{}'.format(station_id, available_dates))
    all_of_the_data.append(retrieveLifetimeData(station_id, available_dates))

imachampion = pd.concat(all_of_the_data, axis=1)
imachampion.to_pickle('currents.pkl')


### Tidal Now

8729149
9414501
9458849
8761742
9440581
8725319
9452749
8575510
9461380
8517756
8761679
8545534
8723999
8720216
8761473
9075079
8726083
8514422
8533935
8728669
9445938
8720221
8720225
8571890
9410650
9412553
8727061
8574821
8766072
8720494
9014096
8779280
8652232
8571181
8516501
8721138
9446451
9468039
8729501
9759197
8722669
8725781
9449639
8777812
8729882
9451805
8723933
8775421
9759412
9414551
8724474
9759421
9456901
8665641
9075059
9063075
8724964
8533541
9452069
8770501
8533941
9442705
9456993
9452318
9452046
9451346
8538875
8748568
9412802
8721994
9414866
8722956
9075034
8722939
9462450
9457287
8530186
9469854
8729197
8516881
8656937
8761724
8726739
8535695
8517276
8724177
8667178
9454411
8532322
9419750
8459479
8537961
8667909
8744934
9414632
8726906
8724353
8551762
9415498
8465705
9415266
9415193
8663665
9490571
8666433
8667623
9446291
8724302
9753216
8674301
9063053
8666653
8467150
8722621
8726573
9410396
8530538
8722487
8725269
8632869
8637589
9462645
9757809
9452400
8535221
